After looking at both CLA and HRP, in my opinion HRP is much better as Markowitz CLA optimal portfolio put most of the extreme weighting in DOGE based on previous expected returns. HRP was much more balanced and allocated most of the money to Bitcoin, Ripple, and Ethereum. MPT's CLA is usually not practical because it is dependent on expected returns for both stocks and cryptocurrencies which are very difficult to be accurate in. This drawback is fixed with HRP. 

I transfered to a jupyter notebook because the code outputs can be seen when uploaded to Github. 

# Heirarchical Risk Parity

In [18]:
######## VaR - Given you used the allocation method
import time
import pandas as pd                           # to work with data frames
import pandas_datareader.data as web          # for the tick price data (yahoo finance)
import datetime as dt                         # datetime to work with dates in the data frames
import numpy as np                            # math for data
from scipy.cluster.hierarchy import linkage   # linkage for distance cluster
from scipy import stats                       # for VaR
import matplotlib.pyplot as plt               # plotting package
%matplotlib inline
import warnings      
warnings.filterwarnings('ignore')             # ignore warnings


### some of the code was taken from Adavances in Financial Machine Learning (Marcos López de Prado)
def returns_distance(period1,period2,ticker_list):
    '''data and tick prices gathered from yahoo finance
    percentage daily returns for each security'''
    df = pd.DataFrame()                                         # dataframe to use during for loop
    failed = []                                                 # to see if any data could not be retrieved
    for crypto in ticker_list:                                   # list of tickers listed
        try:                                                    # try clause just in case it failed
            interval = '1d' 
            query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{crypto}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
            df[crypto] = pd.read_csv(query_string)['Adj Close']
            df[f'{crypto}'] = df[crypto].pct_change()                          # daily return for each portfolio constituent
        except:
            failed.append(crypto)                                             # add failed query ticker to list
            pass
    df = df.dropna()                        # drop null values
    print(f'This script failed retrieving data for: {failed}.')
    corr = df.corr()                       # correlation matrix
    distance_matrix_1 = np.sqrt(0.5*(1-corr))         # distance matrix
    distance = linkage(distance_matrix_1, 'single')   # how the distance is calculated (between the two clusters)
    return distance, df                               # show data frame
    

def matrix_seriation(distance):
    '''Placing the largest covariances together along a diagonal with 
    the smaller covariances around the largest ones'''
    distance = distance.astype(int)                          # sort clustered items by distance
    tracking = pd.Series([distance[-1,0], distance[-1,1]])   # get the first and the second item of the last tuple
    total = distance[-1, 3]                                  # the total num of items is the third item of the last list
    while tracking.max() >= total:                           # if the max of order is bigger than or equal to the max_items
        tracking.index = range(0, tracking.shape[0]*2, 2)    # odd numers as index
        cluster_data = tracking[tracking >= total]           # find clusters
        first = cluster_data.index                           # cluster data contain even index and cluster index
        second = cluster_data.values - total                 
        tracking[first] = distance[second,0]                            
        cluster_data  = pd.Series(distance[second, 1], index=first+1)    
        tracking = tracking.append(cluster_data)            
        tracking = tracking.sort_index()                    
        tracking.index = range(tracking.shape[0])            
    return tracking.tolist()                                


def cluster_variance(covariance, total_m):                # function to be used for the recursive bisection function
    mat_cov = covariance.iloc[total_m, total_m]           # matrix slice
    inverse_variance_port = 1./np.diag(mat_cov)           # calculate the inverse-variance portfolio
    inverse_variance_port/=inverse_variance_port.sum()     
    reshape_inv = inverse_variance_port.reshape(-1,1)     
    cluster_variance_final = np.dot(np.dot(reshape_inv.T, mat_cov), reshape_inv)[0,0]  # 
    return cluster_variance_final                          


def recursive_bisection(covariance, tracking):
    ''''''
    weighting = pd.Series(1, index=tracking)     # intialize weights of 1
    total_m = [tracking]                         # intialize all items in one cluster
    while len(total_m) > 0:               
        total_m = [first[int(second):int(third)] for first in total_m for second,third in 
                   ((0,len(first)/2),(len(first)/2,len(first))) if len(first)>1]
        for first in range(0, len(total_m), 2):  
            a = total_m[first]   
            b = total_m[first+1] 
            v_a = cluster_variance(covariance, a)
            v_b = cluster_variance(covariance, b)
            alpha = 1 - v_a/(v_a+v_b)
            weighting[a] *= alpha
            weighting[b] *=1-alpha
    return weighting


def portfolio_returns(symbols,weights,period1,period2):
    df = pd.DataFrame()
    for crypto in ticker_list:
        interval = '1d' 
        query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{crypto}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
        df[crypto] = pd.read_csv(query_string)['Adj Close']
    df1 = df.apply(lambda x: pd.Series(x.dropna().values))
    returns = ((df1/df1.shift(1))-1)[1:]        # daily return for each security in the portfolio
    port_weighted = (weights * returns)         # apply the weights to each day
    final_return = port_weighted.sum(axis=1)    # sum each of the weights on each day to get the portfolio daily return
    return final_return                 


if __name__ == '__main__':
    aum = input("How much money are you looking to invest in these cryptocurrencies?")  # total amount of money put into the portfolio
    ticker_list = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'ADA-USD', 'BNB-USD', 'DOGE-USD','LINK-USD', 'LTC-USD']
    period1 = int(time.mktime((dt.datetime.now() - dt.timedelta(365*3)).timetuple()))
    period2 = int(time.mktime(dt.datetime.now().timetuple()))
    distance,df = returns_distance(period1,period2,ticker_list) # distance calculated between clusters and portfolio constituent returns
    tracking = matrix_seriation(distance)              
    covariance = df.cov()
    weights = recursive_bisection(covariance, tracking)
    new_index = [df.columns[i] for i in weights.index]
    weights.index = new_index
    portfolio_optimization = weights.sort_values(ascending = False)
    allocation = portfolio_optimization * float(aum)
    final_weights = weights.reindex(ticker_list)
    portfolio_final = portfolio_returns(ticker_list,final_weights,period1,period2)
    mu = np.mean(portfolio_final)                    # mean of portfolio returns
    std = np.std(portfolio_final)                    # standard deviation of portfolio returns
    n_sims = 10000000                                # used large number but not too large to slow things down
    sim_returns = np.random.normal(mu, std, n_sims)  # simulation given mean and standard deviation returns
    SimVAR = float(aum)*np.percentile(sim_returns, 1)       # monte carlo value at risk 
    print('For the portfolio with low volatility...')
    print(allocation)
    print('Simulated Value at Risk is ', SimVAR)
    # heirarchal risk parity 

How much money are you looking to invest in these cryptocurrencies?10000
This script failed retrieving data for: [].
For the portfolio with low volatility...
BTC-USD     2347.553951
XRP-USD     1499.915015
BNB-USD     1365.853001
ETH-USD     1304.318109
LTC-USD     1147.660484
ADA-USD     1055.694313
LINK-USD     946.154352
DOGE-USD     332.850775
dtype: float64
Simulated Value at Risk is  -993.297368019882


# Modern Portfolio Theory - Markowitz (CLA)

In [7]:
import pandas as pd                                        # used for manipulating the dataset
import numpy as np                                         # math uses
import datetime as dt                                      # time for the FRED data manipulation
import pandas_datareader.data as web                       # used for getting the risk free rate for today

data = pd.read_excel('crypto_prices.xlsx').set_index("Date") # reading in the dataset
df = data.pct_change().dropna()                            # daily returns of the portfolio
df = df[:-1]                                               # there was a duplicate
a = 1/len(df.columns)                                      # how many stocks in the potfolio
weight_portfolio = [a,a,a,a,a,a,a,a]                       # equal weighted portfolio
portfolio_return = df.dot(weight_portfolio)                # portfolio return for the whole EW portfolio
variance_matrix = df.cov()*252                             # the amount of trading days in a year
port_var = np.transpose(weight_portfolio)@variance_matrix@weight_portfolio  # portfolio variance
portfolio_vol = np.sqrt(port_var)                                           # portfolio volatility

In [12]:
port_returns = []
port_vol = []
port_weights = []
stocks = len(df.columns)  # number of stocks in given portfolio
portfolios = 1000         # number of portfolios 
data.index = pd.to_datetime(data.index, format='%Y/%m/%d')
ind_rets = data.resample('Y').last().pct_change().mean()  # yearly annual returns of individual securities
for port in range(portfolios):            # 1000 times
    weights = np.random.random(stocks)    # random weights to start
    weights = weights / np.sum(weights)   # standardizing weights to equal to 1
    port_weights.append(weights)          # adding the given weights to be used for portfolio returns
    returns = np.dot(weights,ind_rets)    # portfolio returns
    port_returns.append(returns)          # adding returns to the portfolio returns list to be used for the df
    var = variance_matrix.mul(weights,axis=0).mul(weights,axis=1).sum().sum()
    sd = np.sqrt(var)                     # volatility is the square root of the variance
    ann_sd = (sd*np.sqrt(251))/10         # times it by the 251 returns in a year to make it annual 
    port_vol.append(ann_sd)               # add these items to the list to be used in the data frame and dictionary
main = {"Returns":port_returns,"Volatility":port_vol}
for counter, symbol in enumerate(data.columns.tolist()):
    main[symbol+ 'weight'] = [w[counter] for w in port_weights]
portfolios_V1 = pd.DataFrame(main)
portfolios_V1

,Returns,Volatility,BTC-USDweight,ETH-USDweight,XRP-USDweight,ADA-USDweight,BNB-USDweight,DOGE-USDweight,LINK-USDweight,LTC-USDweight
0,2.018247,1.287304,0.115231,0.120285,0.186918,0.203328,0.180486,0.099416,0.007973,0.086364
1,2.622527,1.448127,0.091628,0.154773,0.116123,0.043750,0.033959,0.233011,0.181143,0.145613
2,2.202247,1.427406,0.056593,0.056988,0.281196,0.141175,0.080126,0.191539,0.018818,0.173565
3,1.870054,1.252928,0.139336,0.102921,0.157157,0.150747,0.179058,0.040925,0.210725,0.019131
4,2.531882,1.398482,0.030211,0.092698,0.068670,0.215267,0.138101,0.116716,0.329406,0.008931
...,...,...,...,...,...,...,...,...,...,...
995,2.482015,1.334049,0.073675,0.181507,0.009111,0.206146,0.152571,0.141726,0.117617,0.117647
996,2.140112,1.309912,0.024027,0.125310,0.111989,0.212186,0.199279,0.075056,0.156715,0.095437
997,2.053202,1.254910,0.241279,0.140481,0.049710,0.061171,0.047438,0.128860,0.206492,0.124571
998,1.979206,1.302305,0.068618,0.071870,0.065973,0.177650,0.126248,0.077338,0.229534,0.182768


# Optimal Portfolio - Tangent Portfolio

In [15]:
start = '2020-01-01'         # using the FRED API for the risk free rate
end = dt.datetime.now()      # datetime of now to get the risk free rate 
risk_free = web.DataReader('DGS10','fred',start,end)  # DGS10 is the 10 year treasury yield
rf = risk_free.iloc[-1]/100  # get the most recent treasury rate  
rf = rf.values               # get the value so I can use it in the sharpe ratio equation
optimal = portfolios_V1.iloc[((portfolios_V1['Returns']-rf)/(portfolios_V1['Volatility']/100)).idxmax()] # highest sharpe ratio
optimal

Returns           3.578081
Volatility        1.602882
BTC-USDweight     0.059009
ETH-USDweight     0.185472
XRP-USDweight     0.040840
ADA-USDweight     0.008810
BNB-USDweight     0.280787
DOGE-USDweight    0.319341
LINK-USDweight    0.016263
LTC-USDweight     0.089478
Name: 883, dtype: float64